Name:Sanika Sunil Kaginkar
Roll No:22102A2008
github link: https://github.com/Sanika-Kaginkar/NLP_LAB/blob/main/NLP_Exp2.ipynb

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import string
import warnings
warnings.filterwarnings('ignore')

In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) - {'not','no'}

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data=pd.read_csv('Reviews.csv')
data = data[:10000]
data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
9995,9996,B000P41A28,A3A63RACXR1XIL,"A. Boodhoo ""deaddodo""",10,15,1,1204502400,constipation,we switched from the advance similac to the or...
9996,9997,B000P41A28,A5VVRGL8JA7R,Adam,2,3,5,1306368000,Constipation Not A Problem if...,"Like the bad reviews say, the organic formula ..."
9997,9998,B000P41A28,A2TGDTJ8YCU6PD,geena77,0,0,5,1347494400,Love this formula!,I wanted to solely breastfeed but was unable t...
9998,9999,B000P41A28,AUV4GIZZE693O,"Susan Coe ""sueysis""",1,2,5,1203638400,very convenient,i love the fact that i can get this delieved t...


In [5]:
data= data.drop(columns=['Id','UserId','ProfileName','Time'])
data

,ProductId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text
0,B001E4KFG0,1,1,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,B00813GRG4,0,0,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,B000LQOCH0,1,1,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,B000UA0QIQ,3,3,2,Cough Medicine,If you are looking for the secret ingredient i...
4,B006K2ZZ7K,0,0,5,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...
9995,B000P41A28,10,15,1,constipation,we switched from the advance similac to the or...
9996,B000P41A28,2,3,5,Constipation Not A Problem if...,"Like the bad reviews say, the organic formula ..."
9997,B000P41A28,0,0,5,Love this formula!,I wanted to solely breastfeed but was unable t...
9998,B000P41A28,1,2,5,very convenient,i love the fact that i can get this delieved t...


In [8]:
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 17.0 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16506 sha256=4d8ffb9fedda9c03982d6b12e83a9573a781a118cdbde65d7b62f6fa056e5e04
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [6]:
import swifter

def preprocess(review):
    review = review.lower()
    review = re.sub(r'[^\w\s]', '', review)
    review = re.sub(r'<[^>]+>', '', review)
    review = re.sub(r'http[s]?://\S+', '', review)
    review = re.sub(r'[\w\.-]+@[\w\.-]+\.\w+', '', review)
    review = re.sub(r'\d+', '', review)
    review = re.sub(r'\s+', ' ', review).strip()
    words = review.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

data['process_text'] = data['Text'].swifter.apply(preprocess)

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

#Stemming

##1. PorterStemmer

In [7]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def preprocess(text):
    words = text.split()
    stem = [stemmer.stem(word) for word in words]
    return ' '.join(stem)

data['porter_stem']=data['process_text'].swifter.apply(preprocess)

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

##2. Snowball Stemmer

In [8]:
from nltk.stem import SnowballStemmer

snowball = SnowballStemmer("english")
def preprocess2(text):
    words = text.split()
    stem = [snowball.stem(word) for word in words]
    return ' '.join(stem)

data['snowball_stem']=data['process_text'].swifter.apply(preprocess2)

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

##3. Lancaster Stemmer

In [9]:
from nltk.stem import LancasterStemmer

lancaster = LancasterStemmer()
def preprocess3(text):
    words = text.split()
    stem = [lancaster.stem(word) for word in words]
    return ' '.join(stem)

data['lancaster_stem']=data['process_text'].swifter.apply(preprocess3)

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

#Lemmatization

##1. Wordnet Lemmatization



In [10]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def preprocess4(text):
    words = text.split()
    lemm = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemm)

data['wordnet_lemm']=data['process_text'].swifter.apply(preprocess4)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

##2. Spacy Lemmatization

In [19]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 62.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
import spacy
nlp = spacy.load("en_core_web_sm")

def preprocess5(text):
  doc = nlp(text)
  lemmatized_text = [token.lemma_ for token in doc]
  return ' '.join(lemmatized_text)

data['spacy_lemm']=data['process_text'].swifter.apply(preprocess5)

Pandas Apply:   0%|          | 0/10000 [00:00<?, ?it/s]

In [15]:
data

,ProductId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text,process_text,porter_stem,snowball_stem,lancaster_stem,wordnet_lemm,spacy_lemm
0,B001E4KFG0,1,1,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,bought several vitality canned dog food produc...,bought sever vital can dog food product found ...,bought sever vital can dog food product found ...,bought sev vit can dog food produc found good ...,bought several vitality canned dog food produc...,buy several vitality can dog food product find...
1,B00813GRG4,0,0,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled jumbo salted peanutsth...,product arriv label jumbo salt peanutsth peanu...,product arriv label jumbo salt peanutsth peanu...,produc ar label jumbo salt peanutsth peanut ac...,product arrived labeled jumbo salted peanutsth...,product arrive label jumbo salt peanutsthe pea...
2,B000LQOCH0,1,1,4,"""Delight"" says it all",This is a confection that has been around a fe...,confection around centuries light pillowy citr...,confect around centuri light pillowi citru gel...,confect around centuri light pillowi citrus ge...,confect around century light pillowy citr gela...,confection around century light pillowy citrus...,confection around century light pillowy citrus...
3,B000UA0QIQ,3,3,2,Cough Medicine,If you are looking for the secret ingredient i...,looking secret ingredient robitussin believe f...,look secret ingredi robitussin believ found go...,look secret ingredi robitussin believ found go...,look secret ingredy robitussin believ found go...,looking secret ingredient robitussin believe f...,look secret ingredient robitussin believe find...
4,B006K2ZZ7K,0,0,5,Great taffy,Great taffy at a great price. There was a wid...,great taffy great price wide assortment yummy ...,great taffi great price wide assort yummi taff...,great taffi great price wide assort yummi taff...,gre taffy gre pric wid assort yummy taffy deli...,great taffy great price wide assortment yummy ...,great taffy great price wide assortment yummy ...
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,B000P41A28,10,15,1,constipation,we switched from the advance similac to the or...,switched advance similac organic product think...,switch advanc similac organ product think woul...,switch advanc similac organ product think woul...,switch adv similac org produc think would bet ...,switched advance similac organic product think...,switch advance similac organic product thinkin...
9996,B000P41A28,2,3,5,Constipation Not A Problem if...,"Like the bad reviews say, the organic formula ...",like bad reviews say organic formula constipat...,like bad review say organ formula constip daug...,like bad review say organ formula constip daug...,lik bad review say org formul constip daught h...,like bad review say organic formula constipate...,like bad review say organic formula constipate...
9997,B000P41A28,0,0,5,Love this formula!,I wanted to solely breastfeed but was unable t...,wanted solely breastfeed unable keep supplemen...,want sole breastfe unabl keep supplement formu...,want sole breastfe unabl keep supplement formu...,want sol breastfee un keep suppl formul chos s...,wanted solely breastfeed unable keep supplemen...,want solely breastfeed unable keep supplement ...
9998,B000P41A28,1,2,5,very convenient,i love the fact that i can get this delieved t...,love fact get delieved house no delievy charge...,love fact get deliev hous no delievi chargeit ...,love fact get deliev hous no delievi chargeit ...,lov fact get deliev hous no delievy chargeit h...,love fact get delieved house no delievy charge...,love fact get delieve house no delievy chargei...


In [16]:
print("Initial Text = ",data['process_text'][0])
print("Porter Stem Text = ",data['porter_stem'][0])
print("Snowball Stem Text = ",data['snowball_stem'][0])
print("Lancaster Stem Text = ",data['lancaster_stem'][0])
print("Wordnet Lemmatized Text = ",data['wordnet_lemm'][0])
print("Spacy Lemmatized Text = ",data['spacy_lemm'][0])

Initial Text =  bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better
Porter Stem Text =  bought sever vital can dog food product found good qualiti product look like stew process meat smell better labrador finicki appreci product better
Snowball Stem Text =  bought sever vital can dog food product found good qualiti product look like stew process meat smell better labrador finicki appreci product better
Lancaster Stem Text =  bought sev vit can dog food produc found good qual produc look lik stew process meat smel bet labrad finicky apprecy produc bet
Wordnet Lemmatized Text =  bought several vitality canned dog food product found good quality product look like stew processed meat smell better labrador finicky appreciates product better
Spacy Lemmatized Text =  buy several vitality can dog food product find good quality product look like stew process meat smell well labrador 